In [1]:
#from nipype.interfaces import niftyreg
import numpy as np
from skimage import io
from pathlib import Path
import re
import ants
from skimage.transform import resize
from tqdm import tqdm
from skimage.morphology import skeletonize_3d, binary_dilation, binary_closing
from scipy.ndimage import distance_transform_edt
import tifffile as tif
from scipy.ndimage import binary_fill_holes
import cc3d
from scipy.io import loadmat, savemat
import skan
import sknw
import networkx as nx
import pickle
import os
import matplotlib.pyplot as plt
import pandas as pd

# Define connected componnet removal

In [2]:
def remove_small_comps_3d(image, thresh = 500):
    """
    

    Parameters
    ----------
    image : binary np array with uint8 elements
        3d numpy matrix, connected components will be removed form this image
    thresh : int64
        smallest connected components to keep

    Returns
    -------
    np.array with uint8 elements, binary
        binary image with connected components below the threshold removed.

    """
    img_lab, N = cc3d.connected_components(image,return_N=True)
    unique, counts = np.unique(img_lab, return_counts=True)
    unique_keep = unique[counts>thresh]
    unique_keep = np.delete(unique_keep,[0])
    img_filt = np.zeros(img_lab.shape).astype('int8')
    img_filt[np.isin(img_lab,unique_keep)] = 1
    return img_filt.astype('uint8')   

def fill_holes(img,thresh=100):
    #res = np.zeros(img.shape)
    for i in np.unique(img)[::-1]:
        _tmp = (img==i)*1.0
        _tmp = _tmp.astype('int8')
        _tmp = remove_small_comps_3d(_tmp,thresh=thresh)
        img[_tmp==1] = i
    res = img.astype('int8')
    return res

# Get mean predictions

In [3]:
directory = Path('matt_preds')
files  = directory.glob('*-*_mean.npy')
files = sorted([x.as_posix() for x in files])
#files = [x for x in files if 'vbm' not in x]

In [4]:
len(files)

767

# High bias low varience segmentation

With removal of connected components under 500 pixels

In [5]:
min_prob = 0.75
max_var = 0.1
for file in tqdm(files[::-1]):
    if not os.path.exists(re.sub('mean','seg',file)):
        mean = np.load(file)
        std = np.load(re.sub('mean','std',file))
        seg = np.zeros(mean.shape[1:])
        seg[(mean[1,:,:,:] > min_prob) * (std[1,:,:,:] < max_var)] = 1
        seg[(mean[2,:,:,:] > min_prob) * (std[2,:,:,:] < max_var)] = 2
        seg = seg.astype('int8')
        seg = (seg==1)*1
        seg = fill_holes(seg)
        np.save(re.sub('mean','seg',file),seg)
        #savemat(re.sub('mean.npy','seg.mat',file),{'FinalImage':fill_holes(binary_dilation(binary_dilation(seg)))})
        #tif.imwrite(re.sub('mean.npy','seg.tif',file),seg)

100%|█████████████████████████████████████████| 767/767 [00:28<00:00, 26.46it/s]


# Get distance transform of neuron segmentation

In [6]:
directory = Path('matt_preds')
files  = directory.glob('*-*_mean.npy')
files = sorted([x.as_posix() for x in files])
files = [x for x in files if 'vbm' not in x]

In [7]:
min_prob = 0.75
max_var = 0.1
for file in tqdm(files[::-1]):
    if not os.path.exists(re.sub('mean','seg_nrn_dst',file)):
        mean = np.load(file)
        std = np.load(re.sub('mean','std',file))
        seg = np.zeros(mean.shape[1:])
        seg[(mean[1,:,:,:] > min_prob) * (std[1,:,:,:] < max_var)] = 1
        seg[(mean[2,:,:,:] > min_prob) * (std[2,:,:,:] < max_var)] = 2
        seg = seg.astype('int8')
        seg = (seg==2)*1
        np.save(re.sub('mean','seg_nrn',file),seg)
        np.save(re.sub('mean','seg_nrn_dst',file),distance_transform_edt(1-seg))

100%|█████████████████████████████████████████| 673/673 [00:14<00:00, 47.41it/s]


In [8]:
#distance_transform_edt(1-seg)

# ANTs registration

## Upsample raw images

In [9]:
image_path = Path('/home/rozakmat/projects/rrg-bojana/data/THY1-TBI')
images = list(image_path.glob('*?[0-9]/*res*?[0-9].tif'))
images = sorted([x.as_posix() for x in images])
len(images)

767

In [10]:
for i in tqdm(range(len(images))[::-1]):
    if not os.path.exists('matt_preds/'+re.sub('/','-',re.sub('/home/rozakmat/projects/rrg-bojana/data/THY1-TBI/','',images[i]))):
        image = io.imread(images[i])
        if image.shape[0] == 96:
            image = np.swapaxes(image,0,1)
            image = np.swapaxes(image,1,3)
            image = resize(image,(2,507,507,252),preserve_range=True)
            zeros = np.zeros((1,507,507,252)).astype('uint16')
            image = np.append(image,zeros,axis = 0)
            image = image.astype('float16')
            io.imsave('matt_preds/'+re.sub('/','-',re.sub('/home/rozakmat/projects/rrg-bojana/data/THY1-TBI/','',images[i])),image)

100%|█████████████████████████████████████████| 767/767 [05:21<00:00,  2.39it/s]


In [56]:
print('matt_preds/'+re.sub('.tif','_upsamp.tif',Path(images[i]).name))
print(images[i])
image.shape[0] == 96

matt_preds/XYZres052_0001_upsamp.tif
/home/rozakmat/projects/rrg-bojana/data/THY1-TBI/vbm11 Apr 04 2020/XYZres052_0001.tif


True

## reshape numpy seg files and resave as tif

In [14]:
directory_seg = Path('matt_preds')
files_seg  = list(directory_seg.glob('*_seg.npy'))
files_seg = sorted([x.as_posix() for x in files_seg])[::-1]

In [16]:
for i in tqdm(range(len(files_seg))):
    if not os.path.exists(re.sub('.npy','.tif',files_seg[i])):
        tmp = np.load(files_seg[i])
        tmp = np.reshape(tmp,(1,507,507,252))
        io.imsave(re.sub('.npy','.tif',files_seg[i]),tmp)

100%|████████████████████████████████████| 1157/1157 [00:00<00:00, 59605.84it/s]


## Register images and transform masks

In [19]:
directory_seg = Path('matt_preds')
images = list(directory_seg.glob('*_0001.tif'))
images = sorted([x.as_posix() for x in images])[::-1]
print(len(images))
print(images[0])

380
matt_preds/vbm11 Apr 04 2020-XYZres052_0001.tif


In [20]:
for i in tqdm(range(len(images))):
    if os.path.exists(re.sub('.tif','_seg.tif',images[i])):
        if not os.path.exists(re.sub('seg','seg_warped',re.sub('.tif','_seg.tif',images[i]))):
            fix = ants.image_read(re.sub('_0001','',images[i]))
            mov = ants.image_read(images[i])
            fix_mask = ants.image_read(re.sub('.tif','_seg.tif',re.sub('_0001','',images[i])))
            mov_mask = ants.image_read(re.sub('.tif','_seg.tif',images[i]))
            mytx = ants.registration(fixed = fix,
                                    moving = mov,
                                    type_of_transform = 'Rigid'
                                    )
            warpedmask = ants.apply_transforms(fixed = fix_mask,
                                               moving = mov_mask,
                                               transformlist = mytx['fwdtransforms'],
                                               interpolator = 'nearestNeighbor'
                                              )
            io.imsave(re.sub('seg','seg_warped',re.sub('.tif','_seg.tif',images[i])),warpedmask.numpy())
            io.imsave(re.sub('seg','seg_warped',re.sub('.tif','_seg.tif',re.sub('_0001','',images[i]))),fix_mask.numpy())
            #print(2 * np.sum(warpedmask.numpy()*fix_mask.numpy())/(np.sum(warpedmask.numpy())+np.sum(fix_mask.numpy())))

 13%|████▋                                | 48/380 [1:23:50<9:39:37, 104.75s/it]/tmp/ipykernel_3840486/2376430453.py:17: UserWarning:

matt_preds/Feb52021_6-XYZres280_0001_seg_warped.tif is a low contrast image

/tmp/ipykernel_3840486/2376430453.py:18: UserWarning:

matt_preds/Feb52021_6-XYZres280_seg_warped.tif is a low contrast image

 13%|████▉                                | 51/380 [1:29:25<9:56:54, 108.86s/it]/tmp/ipykernel_3840486/2376430453.py:17: UserWarning:

matt_preds/Feb52021_6-XYZres277_0001_seg_warped.tif is a low contrast image

/tmp/ipykernel_3840486/2376430453.py:18: UserWarning:

matt_preds/Feb52021_6-XYZres277_seg_warped.tif is a low contrast image

 14%|█████▎                               | 55/380 [1:36:28<9:36:50, 106.49s/it]/tmp/ipykernel_3840486/2376430453.py:17: UserWarning:

matt_preds/Feb52021_6-XYZres270_0001_seg_warped.tif is a low contrast image

/tmp/ipykernel_3840486/2376430453.py:18: UserWarning:

matt_preds/Feb52021_6-XYZres270_seg_warped.tif is a low

## Save Matlab .mat file of registered images

In [ ]:
directory_seg = Path('matt_preds')
images = list(directory_seg.glob('*_0001_seg_warped.tif'))
images = sorted([x.as_posix() for x in images])[::-1]

In [ ]:
for image in tqdm(images):
    if not os.path.exists(re.sub('_seg_warped.tif','_seg_warped_single.mat',re.sub('_0001','',image))):
        img_0001 = io.imread(image)
        img = io.imread(re.sub('_0001','',image))
        seg = img*img_0001
        seg = (seg==1)*1
        seg = seg.astype('int8')
        seg = fill_holes(seg)
        savemat(re.sub('_seg_warped.tif','_seg_warped_single.mat',re.sub('_0001','',image)),{'FinalImage':fill_holes(binary_dilation(seg))})
        
        

# Generate Graphs

In [ ]:
directory = Path('matt_preds')
files_seg_0001 = directory.glob('*_0001_seg_warped.tif')
files_seg_0001 = sorted([x.as_posix() for x in files_seg_0001])
len(files_seg_0001

In [ ]:
for file_0001 in tqdm(files_seg_0001):
    file = file_0001
    skel_file = re.sub('_0001_seg_warped.tif','_skel_warped_single.mat',file)
    #seg = io.imread(file)
    skel = loadmat(skel_file)['FilteredImage']
    #skel = skeletonize_3d(skel)
    io.imsave(re.sub('_seg_warped.tif','_single_skel.tif',file),skel)
    #dst_tsfm = distance_transform_edt(seg)
    #dst_tsfm[dst_tsfm==0]=0.00001
    #skel_dst = skel*dst_tsfm
#
    #
    #np.save(re.sub('seg','dst_skel',file),skel_dst)
    #io.imsave(re.sub('_seg_warped.tif','_dst_skel_warped.tif',file),skel_dst)
    graph = sknw.build_sknw(skel, multi=False)
    print(len(graph.edges))
    #graph, c0  = skan.csr.skeleton_to_csgraph(skel)
    #print(len(graph.edges))
    
    #print(len(graph_0001.edges))
    #pickle.dump(graph, open(str(re.sub('_seg.npy','.pickle',file)), 'w'))
    #nx.write_pajek(graph,re.sub('_seg.npy','.pajek',file))
    
    nx.write_gpickle(graph,re.sub('_0001_seg_warped.tif','_warped.pickle',file))

# write vessel measurments to graph files

In [9]:
directory = Path('matt_preds')
files = directory.glob('*_warped.pickle')
files = sorted([x.as_posix() for x in files])
len(files)

195

In [ ]:
i=0
xls = pd.ExcelFile('TBI_STIM_metalog_local.xlsx')
df = {}
for sheet_name in xls.sheet_names:
    df[sheet_name] = xls.parse(sheet_name)

for file in tqdm(files):
    graph = nx.read_gpickle(file)
    if len(graph.edges) < 1500:
        seg_file = re.sub('_warped.pickle','_seg_warped.tif',file)
        seg_0001_file = re.sub('_warped.pickle','_0001_seg_warped.tif',file)
        seg = io.imread(seg_file)
        seg_0001 = io.imread(seg_0001_file)
        seg_dst = distance_transform_edt(seg)
        seg_0001_dst = distance_transform_edt(seg_0001)
        for sheet_name in xls.sheet_names:
            if re.sub('matt_preds/','',re.sub('_warped.pickle','',file)) in df[sheet_name].values:
                subj = sheet_name
                if subj in ["TBI07_3D","TBI11_3D","TBI22_3D","TBI31_3D","TBI38_3D","SHAM09_3D","SHAM12_3D","SHAM23_3D","SHAM32_3D"]:
                    gender = 'male'
                else:
                    gender = 'female'
                treatment = re.sub('SHA','SHAM',subj[0:3])
                _tmp = df[subj].loc[df[subj]['CHECK WATER'] == re.sub('matt_preds/','',re.sub('_warped.pickle','',file))]
                if _tmp['Unnamed: 12'].iloc[0] == 'raster':
                    wavelength = _tmp['Unnamed: 11'].iloc[0]
                    power_per = _tmp['Unnamed: 10'].iloc[0]
                    start_depth = _tmp['Unnamed: 2'].iloc[0]
                    age = _tmp['Unnamed: 14'].iloc[0]
                    days_post_injury = _tmp['Unnamed: 15'].iloc[0]
                    for i in range(len(graph.edges)):
                        path = graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['pts']
                        _vals = seg_dst[path[::-1,0],path[::-1,1],path[::-1,2]]
                        _vals_0001 = seg_0001_dst[path[::-1,0],path[::-1,1],path[::-1,2]]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii'] = np.mean(_vals)
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii_std'] = np.std(_vals)
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii_0001'] = np.mean(_vals_0001)
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii_0001_std'] = np.std(_vals_0001)
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['delta'] = np.mean(_vals_0001) - np.mean(_vals)
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['gender'] = gender
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['path_weights'] = _vals
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['path_weights_0001'] = _vals_0001
                        #graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['weight'] = graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['weight']
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-0z'] = path[0][0]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-0y'] = path[0][1]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-0x'] = path[0][2]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-1z'] = path[-1][0]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-1y'] = path[-1][1]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-1x'] = path[-1][2]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['img_start_depth'] = start_depth
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['mean_depth'] = np.mean(path[:,0])
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['euclidean-dst'] = np.sqrt(np.sum(np.square(path[-1]-path[0])))
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['subject'] = subj
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['treatment'] = treatment
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['wavelength'] = wavelength
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['power'] = power_per
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['age'] = age
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['days_post_injury'] = days_post_injury
                    nx.write_gpickle(graph, re.sub('preds','preds_graphs',re.sub('.pickle','_radii.pickle',file)))
                    i+=1
print(i)

  1%|▉                                                                                                                                                                           | 1/195 [00:19<1:02:17, 19.26s/it]/tmp/ipykernel_2184055/379099069.py:52: RuntimeWarning:

invalid value encountered in sqrt

 33%|█████████████████████████████████████████████████████████▋                                                                                                                   | 65/195 [23:44<54:37, 25.21s/it]

In [ ]:
_tmp

In [ ]:
path[:,0]

# convert graphs to excel files

In [3]:
directory = Path('matt_preds_graphs')
files = directory.glob('*_warped_radii.pickle')
files = sorted([x.as_posix() for x in files])
print(len(files))
files[0]

141


'matt_preds_graphs/XYZres100_warped_radii.pickle'

In [4]:
for file in tqdm(files):
    graph = nx.read_gpickle(file)
    edge_df = nx.to_pandas_edgelist(graph)
    edge_df.to_excel(re.sub('.pickle','.xlsx',file))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 141/141 [01:21<00:00,  1.73it/s]


In [5]:
edge_df

,source,target,img_start_depth,radii_0001,end-0y,radii,weight,age,radii_0001_std,power,...,euclidean-dst,end-0z,subject,end-0x,treatment,path_weights,path_weights_0001,pts,delta,mean_depth
0,0,42,500,2.408980,392,2.681478,131.381132,221,0.384444,5,...,111.009009,45,TBI11_3D,434,TBI,"[2.449489742783178, 2.23606797749979, 2.449489...","[2.8284271247461903, 2.8284271247461903, 2.828...","[[45, 392, 434], [46, 393, 434], [47, 393, 434...",-0.272498,94.480392
1,1,27,500,2.291214,292,2.165788,73.923346,221,1.089881,5,...,63.395583,59,TBI11_3D,417,TBI,"[4.123105625617661, 3.605551275463989, 4.12310...","[3.7416573867739413, 3.3166247903554, 4.0, 3.6...","[[59, 292, 417], [60, 293, 417], [61, 293, 417...",0.125426,90.500000
2,2,5,500,1.753866,396,2.011129,14.459973,221,0.325225,5,...,12.845233,61,TBI11_3D,479,TBI,"[1.4142135623730951, 2.0, 2.0, 2.0, 2.0, 2.0, ...","[1.4142135623730951, 1.0, 1.7320508075688772, ...","[[61, 396, 479], [62, 396, 480], [63, 397, 481...",-0.257263,65.454545
3,3,28,500,2.486196,497,2.449620,74.776352,221,0.383013,5,...,60.959003,62,TBI11_3D,487,TBI,"[2.8284271247461903, 2.8284271247461903, 2.828...","[2.8284271247461903, 2.0, 2.23606797749979, 2....","[[62, 497, 487], [63, 497, 487], [64, 497, 487...",0.036576,92.000000
4,4,13,500,2.485127,334,2.534549,64.751044,221,0.705417,5,...,44.844175,67,TBI11_3D,437,TBI,"[2.8284271247461903, 2.8284271247461903, 3.0, ...","[3.0, 3.0, 2.8284271247461903, 2.2360679774997...","[[67, 334, 437], [68, 334, 437], [69, 335, 437...",-0.049422,81.693878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,85,85,500,2.759306,387,2.307209,2.828427,221,0.229998,5,...,1.414214,212,TBI11_3D,430,TBI,"[2.23606797749979, 2.449489742783178, 2.236067...","[3.0, 2.8284271247461903, 2.449489742783178]","[[212, 387, 430], [213, 387, 429], [212, 388, ...",0.452097,212.333333
89,86,86,500,2.000000,115,2.000000,3.414214,221,0.000000,5,...,0.000000,215,TBI11_3D,382,TBI,"[2.0, 2.0, 2.0, 2.0]","[2.0, 2.0, 2.0, 2.0]","[[215, 115, 382], [214, 115, 382], [215, 115, ...",0.000000,214.750000
90,88,88,500,3.094899,496,2.621320,4.146264,221,0.414246,5,...,0.000000,220,TBI11_3D,222,TBI,"[2.8284271247461903, 2.8284271247461903, 2.0, ...","[3.1622776601683795, 3.605551275463989, 2.4494...","[[220, 496, 222], [220, 497, 223], [221, 496, ...",0.473579,220.250000
91,89,89,500,2.702115,471,2.828427,3.146264,221,0.178633,5,...,1.000000,222,TBI11_3D,482,TBI,"[2.8284271247461903, 2.8284271247461903, 2.828...","[2.8284271247461903, 2.449489742783178, 2.8284...","[[222, 471, 482], [223, 470, 481], [223, 471, ...",-0.126312,222.666667
